Note that data is already downloaded from Kacper's work.

In [1]:
import pandas as pd
import os, json
from tqdm.notebook import tqdm

tqdm.pandas()

In [2]:
files = os.listdir("../../../data/irrelevant/")

data = {}
errors = []

for i,file in enumerate(tqdm(files)):
    with open('../../../data/irrelevant/' + file,'r') as f:
        try:
            data[i] = json.loads(f.read())
        except:
            errors.append(file)
            continue

data = pd.DataFrame.from_dict(data, orient='index',columns=['URL', 'ExtractedTextUntokenized','RawData'])
data.head()

  0%|          | 0/23733 [00:00<?, ?it/s]

,URL,ExtractedTextUntokenized,RawData
0,http://atom.prattsi.org/index.php/informationo...,\nPratt Institute School of Information On-Sit...,None
1,http://all-comic.com/tag/valiant-universe/,\nValiant Universe Archives - MangaMavericks.c...,None
2,http://business.sweetwaterreporter.com/sweetwa...,\nLatest news about BZI/Recaps - Stock Market ...,None
3,http://apa.hk/fleetdetail.php?fleet_id=2013060...,\nENGLISH|简体中文|繁體中文\nABOUT APA SERVICES FLEET ...,None
4,http://cmahabitat.be/index.php/projets,\nProjets\nLayout Style\nWide\nBoxed\nFramed\n...,None


In [3]:
for file in errors:
    try:
        os.remove('data/irrelevant/' + file)
    except FileNotFoundError:
        continue
print('Removed', len(errors), 'broken files.')

Removed 84 broken files.


In [4]:
# get record where ExtractedTextUntokenized is empty or RawData has a value
pdfs = data[(data['ExtractedTextUntokenized'].isnull())]
pdfs.head()

,URL,ExtractedTextUntokenized,RawData
4281,http://www.altoslashormigas.com/wp-content/upl...,None,%PDF-1.5\r%����\r\n7 0 obj\r<</Linearized 1/L ...
11740,https://api.repository.cam.ac.uk/server/api/co...,None,%PDF-1.7\r%����\r\n2907 0 obj\r<</Linearized 1...
16429,https://api.repository.cam.ac.uk/server/api/co...,None,%PDF-1.5\r%����\r\n1498 0 obj\r<</Linearized 1...
17688,https://altonray.com/wp-content/uploads/2024/0...,None,%PDF-1.5\r%����\r\n459 0 obj\r<</Linearized 1/...
20260,https://www.ctc.cam.ac.uk/stephen70/talks/swh7...,None,%PDF-1.4\n%�����\n1 0 obj\n<< /Type /Catalog\n...


In [5]:
import pymupdf
import requests

def extract_pdf_text(url):
    r = requests.get(url)
    try:
        pdf = pymupdf.open(stream=r.content, filetype='pdf')
    except Exception as e:
        print('Error opening PDF: ', url)
        print(e)
        return None
    return '\n\n'.join([pdf.get_page_text(i) for i in range(len(pdf))])

In [6]:
pdfs['ExtractedTextUntokenized'] = pdfs['URL'].progress_apply(extract_pdf_text)

  0%|          | 0/21 [00:00<?, ?it/s]

Error opening PDF:  http://www.altoslashormigas.com/wp-content/uploads/2018/10/2016-Malbec-Terroir-UCO-Valley-S-T.pdf
code=7: no objects found


Error opening PDF:  http://www.altoslashormigas.com/wp-content/uploads/2018/10/2017-Malbec-Clasico-S-T.pdf
code=7: no objects found


C:\Users\shrey\AppData\Local\Temp\ipykernel_7296\2795950526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdfs['ExtractedTextUntokenized'] = pdfs['URL'].progress_apply(extract_pdf_text)


In [7]:
data.update(pdfs)
data.dropna(subset=['ExtractedTextUntokenized'], inplace=True)
# drop empty strings too
data = data[data['ExtractedTextUntokenized'] != '']

In [8]:

# renaming columns
write_data = data[['URL', 'ExtractedTextUntokenized']]
write_data.columns = ['url', 'text']
write_data['class'] = 'irrelevant'

write_data

,url,text,class
0,http://atom.prattsi.org/index.php/informationo...,\nPratt Institute School of Information On-Sit...,irrelevant
1,http://all-comic.com/tag/valiant-universe/,\nValiant Universe Archives - MangaMavericks.c...,irrelevant
2,http://business.sweetwaterreporter.com/sweetwa...,\nLatest news about BZI/Recaps - Stock Market ...,irrelevant
3,http://apa.hk/fleetdetail.php?fleet_id=2013060...,\nENGLISH|简体中文|繁體中文\nABOUT APA SERVICES FLEET ...,irrelevant
4,http://cmahabitat.be/index.php/projets,\nProjets\nLayout Style\nWide\nBoxed\nFramed\n...,irrelevant
...,...,...,...
23727,https://tenglong.eng.cam.ac.uk/files/tpel-obc-...,0885-8993 (c) 2019 IEEE. Personal use is permi...,irrelevant
23728,https://www.darwin.cam.ac.uk/wp-content/upload...,FOR THE YEAR 2023\nSUSTAINABILITY\nPROGRESS RE...,irrelevant
23729,https://tenglong.eng.cam.ac.uk/files/tpel-nano...,0885-8993 (c) 2019 IEEE. Personal use is permi...,irrelevant
23731,https://tenglong.eng.cam.ac.uk/files/rcp.pdf,This article has been accepted for publication...,irrelevant


In [9]:
# write to json
write_data.to_json('../../../data/labelled/irrelevant.json', orient='records', indent=4)

